In [1]:
!nvidia-smi

Mon Oct  7 15:39:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:

!pip install numpy matplotlib

In [3]:
# Write CUDA kernel for Conway's Game of Life
cuda_code = """
extern "C"
__global__ void update(int *current, int *next, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int alive_neighbors = 0;
        for (int dx = -1; dx <= 1; dx++) {
            for (int dy = -1; dy <= 1; dy++) {
                if (dx == 0 && dy == 0) continue;
                int nx = (x + dx + width) % width;
                int ny = (y + dy + height) % height;
                alive_neighbors += current[ny * width + nx];
            }
        }

        int idx = y * width + x;
        if (current[idx] == 1) {
            next[idx] = (alive_neighbors == 2 || alive_neighbors == 3) ? 1 : 0;
        } else {
            next[idx] = (alive_neighbors == 3) ? 1 : 0;
        }
    }
}
"""

# Save the CUDA code to a file
with open('game_of_life.cu', 'w') as f:
    f.write(cuda_code)

In [4]:
!nvcc game_of_life.cu -o game_of_life

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status


In [5]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660544 sha256=2b6b826abf35b63f3ddd8ac5f4b8ec3344a675df343bf25f1da5a248acc2a11e
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit  # Automatically initializes CUDA context

# Define the CUDA kernel for Conway's Game of Life
cuda_code = """
extern "C"
__global__ void update(int *current, int *next, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int alive_neighbors = 0;
        for (int dx = -1; dx <= 1; dx++) {
            for (int dy = -1; dy <= 1; dy++) {
                if (dx == 0 && dy == 0) continue; // Skip the current cell
                int nx = (x + dx + width) % width; // Wrap around horizontally
                int ny = (y + dy + height) % height; // Wrap around vertically
                alive_neighbors += current[nx + ny * width];
            }
        }

        int current_state = current[x + y * width];
        if (current_state == 1 && (alive_neighbors < 2 || alive_neighbors > 3)) {
            next[x + y * width] = 0; // Cell dies
        } else if (current_state == 0 && alive_neighbors == 3) {
            next[x + y * width] = 1; // Cell becomes alive
        } else {
            next[x + y * width] = current_state; // Stays the same
        }
    }
}
"""

# Initialize parameters
width, height = 100, 100  # Dimensions of the grid

# Initialize the grid with random values (0s and 1s)
initial_grid = np.random.randint(2, size=(width, height)).astype(np.int32)

# Allocate memory on the GPU
current_gpu = cuda.mem_alloc(initial_grid.nbytes)
next_gpu = cuda.mem_alloc(initial_grid.nbytes)

# Copy the initial grid to the GPU
cuda.memcpy_htod(current_gpu, initial_grid)

# Define block and grid sizes
threads_per_block = (16, 16)  # Number of threads in each block
blocks_per_grid = (width // threads_per_block[0] + 1, height // threads_per_block[1] + 1)

# Compile the CUDA kernel
mod = SourceModule(cuda_code)
update = mod.get_function("update")

# Set up a stopping condition
stop_simulation = False

# Display the grid using Matplotlib in a loop
try:
    while not stop_simulation:  # Run until stop_simulation is set to True
        next_grid = np.zeros_like(initial_grid)  # Initialize the next grid
        cuda.memcpy_htod(next_gpu, next_grid)  # Copy the next grid to GPU

        # Launch the CUDA kernel
        update(current_gpu, next_gpu, np.int32(width), np.int32(height),
               block=(threads_per_block[0], threads_per_block[1], 1),
               grid=blocks_per_grid)

        # Copy the next grid back to CPU
        cuda.memcpy_dtoh(next_grid, next_gpu)

        # Visualize the grid using Matplotlib
        clear_output(wait=True)
        plt.imshow(next_grid, cmap='binary')  # Show the grid
        plt.title('Conway\'s Game of Life')
        plt.axis('off')  # Hide the axes
        plt.show()

        # Swap grids for the next iteration
        current_gpu, next_gpu = next_gpu, current_gpu

except KeyboardInterrupt:
    print("Simulation stopped by user.")

except Exception as e:
    print(f"An error occurred: {e}")

# Finally, ensure that resources are cleaned up if needed
print("Exiting simulation.")

Simulation stopped by user.
Exiting simulation.
Error in callback <function _draw_all_if_interactive at 0x7e984569cee0> (for post_execute):